In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

# df = pd.read_csv('/content/gdrive/MyDrive/nlp_project/nlp_project_data/dev.csv')



df = pd.read_csv('/dj/data/dev.csv')



In [2]:
df, eval_df = train_test_split(df, test_size=0.01, random_state=42)

In [3]:
eval_df

,fname,dialogue,summary,topic
460,dev_459,"#Person1#: B씨, 그 손에 들고 있는 건 뭐예요?\r\n#Person2#:...","B는 스포츠 정보를 읽음으로써 압박을 해결하고, 프레드는 피곤할 때 바에 가서 와인...",압박 해결
73,dev_73,#Person1#: 지난 몇 달 동안 예상 밖의 서비스 정지가 너무 많았습니다. 우...,#Person1#은 서비스 중단 시간 동안의 생산량 손실을 줄이기 위해 유지보수 절...,정지 상황 대처
231,dev_231,"#Person1#: 그럼, 문제가 무엇인지 말해줄 수 있을까, 바비?\n#Perso...","바비는 카르다노 박사에게 오른발에 심한 통증이 있다고 말한다.\n오른발을 본 후, ...",의사를 찾아가다
175,dev_175,#Person1#: 직원의 업무가 불만족스러울 때 관리자가 어떻게 대응해야 한다고 ...,#Person1#과 #Person2#가 관리자는 직원의 불만족스러운 업무에 대해 일...,사적인 대화
237,dev_237,"#Person1#: 릴리, 너 리지를 알아? \n#Person2#: 어떤 리지 말이...",#Person1#은 릴리에게 리지의 가족에 대해 묻는다. 왜냐하면 #Person1#...,가족


In [4]:
# 필요한 라이브러리 임포트
import pandas as pd
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
import bitsandbytes as bnb
from evaluate import load

# 데이터 로드
# df = pd.read_csv('/dj/data/train.csv')
# eval_df = pd.read_csv('/dj/data/dev.csv')

# 데이터셋을 Hugging Face의 Dataset 형식으로 변환
train_dataset = Dataset.from_pandas(df)
eval_dataset = Dataset.from_pandas(eval_df)

# 1. 토크나이저 로드 및 특수 토큰 추가
tokenizer = AutoTokenizer.from_pretrained("yanolja/EEVE-Korean-Instruct-10.8B-v1.0")


# 2. 8-bit로 모델 로드 및 토큰 크기 조정
model = AutoModelForCausalLM.from_pretrained(
    "yanolja/EEVE-Korean-Instruct-10.8B-v1.0",
    load_in_8bit=True,
    device_map="auto"
)
model.resize_token_embeddings(len(tokenizer))

# 3. LoRA 설정
lora_config = LoraConfig(
    r=8,                # LoRA 차원
    lora_alpha=16,       # LoRA의 alpha
    lora_dropout=0.1,    # 드롭아웃 비율
    bias="none",         # LoRA가 bias를 학습하지 않음
    task_type="CAUSAL_LM"
)

# 4. 모델에 LoRA 적용
model = get_peft_model(model, lora_config)

# 5. 훈련 설정
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=1,  # A100에 맞춘 적당한 배치 사이즈
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=2,
    num_train_epochs=1,
    logging_dir="./logs",
    learning_rate=1e-4,
    fp16=True,   # 16-bit 훈련으로 메모리 최적화
)


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA SETUP: CUDA runtime path found: /opt/conda/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 118
CUDA SETUP: Loading binary /opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda118.so...


/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [5]:


# 6. 프롬프트 준비
# prompt_template = "다음 대화를 20퍼센트 정도로 요약해 주세요:\n\n{prompt}\n\n요약:"

def preprocess_function(examples):
    # inputs = [prompt_template.format(prompt=dialogue) for dialogue in examples['dialogue']]
    model_inputs = tokenizer(examples['dialogue'], max_length=512, truncation=True, return_tensors="pt", padding="max_length")

    # 레이블의 max_length를 입력과 동일하게 설정
    labels = tokenizer(examples['summary'], max_length=512, truncation=True, return_tensors="pt", padding="max_length").input_ids

    # -100은 CrossEntropyLoss에서 무시되는 토큰 값
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs['labels'] = labels

    return model_inputs

# 데이터 전처리
train_dataset = train_dataset.map(preprocess_function, batched=True)
eval_dataset = eval_dataset.map(preprocess_function, batched=True)

# 7. Rouge 평가 메트릭 로드
rouge = load("rouge")

def compute_metrics(eval_preds):
    predictions, labels = eval_preds
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge 평가
    result = rouge.compute(predictions=decoded_preds, references=decoded_labels)
    return result

# 8. Trainer 설정 및 학습
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# 9. 학습 실행
trainer.train()

# 10. 평가 실행
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

Map:   0%|          | 0/494 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)
Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lindsay-heo (lindsay-heo-upstage). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 56.00 MiB. GPU 0 has a total capacity of 23.69 GiB of which 42.81 MiB is free. Process 2549576 has 23.64 GiB memory in use. Of the allocated memory 20.72 GiB is allocated by PyTorch, and 2.60 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [7]:
# prompt_template = "다음 대화를 20퍼센트 정도로 요약해 주세요:\n\n{prompt}\n\n요약만 출력:"
text = '''#Person1#: 안녕하세요, 오늘 하루 어떠셨어요? 
#Person2#: 요즘 숨쉬기가 좀 힘들어요.
#Person1#: 최근에 감기 같은 것에 걸리신 적이 있나요?
#Person2#: 아니요, 감기는 아니에요. 그냥 숨을 쉴 때마다 가슴이 무겁게 느껴져요.
#Person1#: 알고 있는 알레르기가 있나요?
#Person2#: 아니요, 알고 있는 알레르기는 없어요.
#Person1#: 이런 증상이 항상 나타나나요, 아니면 활동할 때 주로 나타나나요?
#Person2#: 운동을 할 때 많이 나타나요.
#Person1#: 저는 당신을 폐 전문의에게 보내서 천식에 대한 검사를 받게 할 거예요.
#Person2#: 도와주셔서 감사합니다, 의사 선생님'''
model_inputs = tokenizer(text, return_tensors='pt').to('cuda')

outputs = model.generate(**model_inputs, max_new_tokens=256)
output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print(output_text)

Setting `pad_token_id` to `eos_token_id`:32000 for open-end generation.


#Person1#: 안녕하세요, 오늘 하루 어떠셨어요? 
#Person2#: 요즘 숨쉬기가 좀 힘들어요.
#Person1#: 최근에 감기 같은 것에 걸리신 적이 있나요?
#Person2#: 아니요, 감기는 아니에요. 그냥 숨을 쉴 때마다 가슴이 무겁게 느껴져요.
#Person1#: 알고 있는 알레르기가 있나요?
#Person2#: 아니요, 알고 있는 알레르기는 없어요.
#Person1#: 이런 증상이 항상 나타나나요, 아니면 활동할 때 주로 나타나나요?
#Person2#: 운동을 할 때 많이 나타나요.
#Person1#: 저는 당신을 폐 전문의에게 보내서 천식에 대한 검사를 받게 할 거예요.
#Person2#: 도와주셔서 감사합니다, 의사 선생님 # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # # #
